In [3]:
# %load 1_loading.py
import openpyxl
from openpyxl import Workbook

# get workbook object; the data_only option captures the current value of any formulae
# Your file should be in the same directory as this code, OR you need to provide the FULL path to that file, not just it's name
wb = openpyxl.load_workbook('pyxl_demo_pyconcan.xlsx', data_only= True)

# WB -> WS -> Cell
demo_worksheet = wb.get_sheet_by_name("clean_data")

In [4]:
# %load 2_get_sheet.py
# I moved this code to 1_loading.py and was too lazy to renumber

In [5]:
# %load 3_get_employee_list_for_loop.py
employee_ids = set()
for row in demo_worksheet.rows:
    if row[0].value != 'employee_num':
        employee_ids.add(row[0].value)

In [6]:
# %load 4_create_employee_dictionary.py
employee_aggregate = {}
for employee in employee_ids:
    # list comprehension: I'll want to sum the hours later
    hours = [row[6].value for row in demo_worksheet.rows if employee == row[0].value]
    # set comprehension: each employee should belong to only ONE cost center
    cost_center = {row[1].value for row in demo_worksheet.rows if employee == row[0].value}
    division = {str(row[2].value) for row in demo_worksheet.rows if employee == row[0].value}
    manager = {row[3].value for row in demo_worksheet.rows if employee == row[0].value}
    
    assert len(cost_center) == len(division) == len(manager) == 1
    
    employee_aggregate[employee]={
        "hours": sum(hours),
        "cost_center": list(cost_center)[0],
        "division": list(division)[0],
        "manager": list(manager)[0]
    }

In [7]:
# %load 5_create_output_spreadsheet.py
# Create output workbook, then output worksheet
output_book = Workbook() #OpenPyXL object
output_sheet = output_book.create_sheet("Aggregate Time",0)

# Building the Output Header: Specific Cell References
header = [
    demo_worksheet["A1"].value,
    demo_worksheet["B1"].value,
    demo_worksheet["C1"].value,
    demo_worksheet["D1"].value,
    demo_worksheet["G1"].value
]

# Create output data construct & append header 
output_data = []
output_data.append(header)

# Building new rows, then appending them to output data construct
for employee in employee_aggregate: #iterating over dictionary keys
    new_row = []
    new_row.append(employee)
    new_row.append(employee_aggregate[employee]['cost_center'])
    new_row.append(employee_aggregate[employee]['division'])
    new_row.append(employee_aggregate[employee]['manager'])
    new_row.append(employee_aggregate[employee]['hours'])
    output_data.append(new_row)


In [8]:
# %load 6_write_data_to_workbook.py
# Write data to sheet object
for row in output_data:
    rowIn = output_data.index(row)
    for col in range(len(output_data[0])):
        # Adding 1 because spreadsheets count from 1, not 0
        output_sheet.cell(row = rowIn+1, column = col+1).value = output_data[rowIn][col]

# You don't have an actual spreadsheet until you do this
output_book.save (filename = "done_pyconcan.xlsx")

In [9]:
# %load 7_create_json_object.py
# Yes, this import is normally done earlier
import json
with open("aggregate_time.json", 'w') as f:
    json.dump(employee_aggregate, f, sort_keys=True, indent=4)